### Model Review chatbot
Ref: modified from YT Chroma DB Multi doc retriever Langchain Part1.ipynb

By Sybil Shi

In [82]:
!pip -q install langchain openai tiktoken chromadb pypdf gradio

In [ ]:
!pip show langchain

Name: langchain
Version: 0.0.177
Summary: Building applications with LLMs through composability
Home-page: https://www.github.com/hwchase17/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, async-timeout, dataclasses-json, numexpr, numpy, openapi-schema-pydantic, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


In [ ]:
# !wget -q https://www.dropbox.com/s/vs6ocyvpzzncvwh/new_articles.zip
# !unzip -q new_articles.zip -d new_articles

# LangChain multi-doc retriever with ChromaDB

***New Points***
- Multiple Files
- ChromaDB
- Source info 
- gpt-3.5-turbo API

## Setting up LangChain 


In [83]:
import os

os.environ["OPENAI_API_KEY"] = ""

In [84]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
from langchain.document_loaders import TextLoader, PyPDFLoader
from langchain.document_loaders import DirectoryLoader


## Load multiple and process documents

In [85]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [86]:
# Load and process the text files
# loader = TextLoader('single_text_file.txt')
loader = DirectoryLoader('/content/drive/MyDrive/LLM/data/', glob="./*.pdf", loader_cls=PyPDFLoader)
documents = loader.load()

In [ ]:
# Show the documents in the folder
pdf_folder_path = '/content/drive/MyDrive/LLM/data/'
os.listdir(pdf_folder_path)

['fil17022a.pdf',
 'deloitte_model-risk-management_plaquette.pdf',
 'pub-ch-model-risk.pdf',
 'pub-ch-regulatory-reporting.pdf',
 'FAQ on the Targeted Review of Internal Models.pdf']

In [ ]:
# PyPDFLoader("./content/drive/MyDrive/LLM/data/fil17022a.pdf")

# from pypdf import PdfReader
# PdfReader("/content/drive/MyDrive/LLM/data/fil17022a.pdf")
type(documents)

list

In [ ]:
#splitting the text into
# TODO: experinemt with chunk size & overlap
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)

In [ ]:
len(texts)

734

In [ ]:
texts[3]

Document(page_content='management of model risk.  \n  \nThis guidance describes the key aspects of effective model risk management. Section II explains the \npurpose and scope of the guidance, and Section III gives an overview of model risk management. \n                                                            \n1 Unless otherwise indicated, banks  refers to state non -member  banks , state savings associations,  and all other \ninstitutions for which the Federal Deposit Insurance Corporation  is the primary supervisor.   It is not expected that  \nthis guidance will pertain to FDIC -supervised institutions with under $1 billion in total assets  unless the institution’s  \nmodel use is significant, complex, or poses elevated risk to the institution.', metadata={'source': '/content/drive/MyDrive/LLM/data/fil17022a.pdf', 'page': 0})

## create the DB

In [87]:
# Embed and store the texts
# Supplying a persist_directory will store the embeddings on disk
persist_directory = 'db'

## here we are using OpenAI embeddings but in future we will swap out to local embeddings
embedding = OpenAIEmbeddings()

vectordb = Chroma.from_documents(documents=texts, 
                                 embedding=embedding,
                                 persist_directory=persist_directory)

In [88]:
# persiste the db to disk
vectordb.persist()
vectordb = None

In [89]:
# Now we can load the persisted database from disk, and use it as normal. 
vectordb = Chroma(persist_directory=persist_directory, 
                  embedding_function=embedding)

## Make a retriever

In [90]:
retriever = vectordb.as_retriever()

In [91]:
# docs = retriever.get_relevant_documents("How much money did Pando raise?")
docs = retriever.get_relevant_documents("what is a model?")

In [ ]:
len(docs)

4

In [92]:
retriever = vectordb.as_retriever(search_kwargs={"k": 2})

In [93]:
retriever.search_type

'similarity'

In [94]:
retriever.search_kwargs

{'k': 2}

## Make a conversational chain using "ConversationalRetrievalChain"

In [95]:
import os
import openai
openai.api_key = os.getenv("OPENAI_API_KEY")
# openai.Model.list()

In [ ]:
model_lst = openai.Model.list()

for i in model_lst['data']:
    print(i['id'])

whisper-1
babbage
gpt-3.5-turbo
davinci
text-davinci-edit-001
text-davinci-003
babbage-code-search-code
text-similarity-babbage-001
code-davinci-edit-001
text-davinci-001
ada
babbage-code-search-text
babbage-similarity
code-search-babbage-text-001
text-curie-001
code-search-babbage-code-001
text-ada-001
text-embedding-ada-002
text-similarity-ada-001
curie-instruct-beta
ada-code-search-code
ada-similarity
code-search-ada-text-001
text-search-ada-query-001
davinci-search-document
ada-code-search-text
text-search-ada-doc-001
davinci-instruct-beta
text-similarity-curie-001
code-search-ada-code-001
ada-search-query
text-search-davinci-query-001
curie-search-query
davinci-search-query
babbage-search-document
ada-search-document
text-search-curie-query-001
text-search-babbage-doc-001
curie-search-document
text-search-curie-doc-001
babbage-search-query
text-babbage-001
text-search-davinci-doc-001
text-search-babbage-query-001
curie-similarity
curie
gpt-3.5-turbo-0301
text-similarity-davinci-00

In [ ]:
# Example OpenAI Python library request
MODEL = "gpt-3.5-turbo"
# MODEL = "gpt-4"
response = openai.ChatCompletion.create(
    model=MODEL,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Knock knock."},
        {"role": "assistant", "content": "Who's there?"},
        {"role": "user", "content": "Orange."},
    ],
    temperature=0,
)

response

<OpenAIObject chat.completion id=chatcmpl-7INLIwNwVa2BrTeMo0x3NACac0J5j at 0x7f769b5d4680> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "Orange who?",
        "role": "assistant"
      }
    }
  ],
  "created": 1684613988,
  "id": "chatcmpl-7INLIwNwVa2BrTeMo0x3NACac0J5j",
  "model": "gpt-3.5-turbo-0301",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 3,
    "prompt_tokens": 39,
    "total_tokens": 42
  }
}

In [96]:
from langchain.chat_models import ChatOpenAI
# llm = ChatOpenAI(temperature=0,model_name="gpt-4")
llm = ChatOpenAI(temperature=0,model_name="gpt-3.5-turbo")

In [97]:
# Initialise Langchain - Conversation Retrieval Chain
qa = ConversationalRetrievalChain.from_llm(ChatOpenAI(temperature=0), vectordb.as_retriever())

## Front end

In [ ]:
import gradio as gr
with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    msg = gr.Textbox()
    clear = gr.Button("Clear")
    chat_history = []
    
    def user(user_message, history):
        # Get response from QA chain
        response = qa({"question": user_message, "chat_history": history})
        # Append user message and response to chat history
        history.append((user_message, response["answer"]))
        return gr.update(value=""), history
    msg.submit(user, [msg, chatbot], [msg, chatbot], queue=False)
    clear.click(lambda: None, None, chatbot, queue=False)

if __name__ == "__main__":
    demo.launch(debug=True, share=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://34b9baab43364e2de8.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://34b9baab43364e2de8.gradio.live


## Make a Converstional chain using "RetrievalQA"

In [98]:
# Set up the turbo LLM
turbo_llm = ChatOpenAI(
    temperature=0,
    model_name='gpt-3.5-turbo'
)

In [99]:
# create the chain to answer questions 
qa_chain = RetrievalQA.from_chain_type(llm=turbo_llm, 
                                  chain_type="stuff", 
                                  retriever=retriever, 
                                  return_source_documents=True)

In [100]:
## Cite sources
def process_llm_response(llm_response):
    print(llm_response['result'])
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

In [ ]:
# full example
query = "What is a model?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

A model is a simplified representation of real-world relationships among observed characteristics, values, and events. It is used to focus attention on particular aspects considered to be most important for a given model application. Model quality can be measured in many ways, and models are never perfect. The appropriate metrics of quality, and the effort that should be put into improving quality, depend on the situation.


Sources:
/content/drive/MyDrive/LLM/data/pub-ch-model-risk.pdf
/content/drive/MyDrive/LLM/data/fil17022a.pdf


## Try checklist
Ref: https://github.com/marcotcr/checklist

In [101]:
!pip -q install checklist

In [102]:
!jupyter nbextension install --py --sys-prefix checklist.viewer
!jupyter nbextension enable --py --sys-prefix checklist.viewer

Installing /usr/local/lib/python3.10/dist-packages/checklist/viewer/static -> viewer
Up to date: /usr/share/jupyter/nbextensions/viewer/bundle.js
Up to date: /usr/share/jupyter/nbextensions/viewer/bundle.js.map
Up to date: /usr/share/jupyter/nbextensions/viewer/index.js.map
Up to date: /usr/share/jupyter/nbextensions/viewer/custom.js
Up to date: /usr/share/jupyter/nbextensions/viewer/extension.js
Up to date: /usr/share/jupyter/nbextensions/viewer/index.js
Up to date: /usr/share/jupyter/nbextensions/viewer/__init__.py
Up to date: /usr/share/jupyter/nbextensions/viewer/__pycache__/__init__.cpython-310.pyc
- Validating: OK

    To initialize this nbextension in the browser every time the notebook (or other app) loads:
    
          jupyter nbextension enable checklist.viewer --py --sys-prefix
    
Enabling notebook extension viewer/extension...
Paths used for configuration of notebook: 
    	/usr/etc/jupyter/nbconfig/notebook.json
Paths used for configuration of notebook: 
    	
      - 

In [ ]:
# torch.__version__
!pip show torch

Name: torch
Version: 2.0.1+cu118
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3
Location: /usr/local/lib/python3.10/dist-packages
Requires: filelock, jinja2, networkx, sympy, triton, typing-extensions
Required-by: fastai, sentence-transformers, torchaudio, torchdata, torchtext, torchvision, triton


In [ ]:
!pip show tensorflow

Name: tensorflow
Version: 2.12.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: absl-py, astunparse, flatbuffers, gast, google-pasta, grpcio, h5py, jax, keras, libclang, numpy, opt-einsum, packaging, protobuf, setuptools, six, tensorboard, tensorflow-estimator, tensorflow-io-gcs-filesystem, termcolor, typing-extensions, wrapt
Required-by: dopamine-rl


In [103]:
!python -m spacy download en_core_web_sm

2023-05-22 21:05:54.805596: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 42.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


### MFT - Language prompt with accurate response
- There are other more basic functions I could test, i.e., to check if the model can understand I am asking for a review team, not an lob. But I require the model to respond with the correct content. This requires users to privde the correct answer. By using the `meta=True` argument for `editor.template()`, the corrrent answers associated with the prompt will be will be stored in the `answers_prompts` object.
- This process can be used to write test cases for 9 Answer types, I am doing one for answer type 'Number'. 
-[ ] TODO: write test cases for rest 8 answer types

In [105]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import pandas as pd
import random
import checklist
from checklist.editor import Editor
from checklist.expect import Expect
from checklist.pred_wrapper import PredictorWrapper

from typing import List
import warnings
warnings.filterwarnings('ignore')

import spacy
import itertools


import checklist.text_generation
from checklist.test_types import MFT, INV, DIR

import numpy as np

from checklist.test_suite import TestSuite
from checklist.perturb import Perturb


In [106]:
editor = Editor()

In [107]:
number_prompts = editor.template("The number of testing for {model_type} is  ", model_type = ["marketing model", "CCAR model"], meta=True)
correct_responses = {
    "marketing model": "2",
    "CCAR model": "3"
}

In [108]:
type(number_prompts.data)

list

In [109]:
number_prompts.data

['The number of testing for marketing model is  ',
 'The number of testing for CCAR model is  ']

In [110]:
number_prompts.meta

[{'model_type': 'marketing model'}, {'model_type': 'CCAR model'}]

In [111]:
qa_chain('The number of testing for marketing model is')

{'query': 'The number of testing for marketing model is',
 'result': 'The text does not provide information on the specific number of testing for marketing models. It only states that testing should be applied to actual circumstances under a variety of market conditions, including scenarios that are outside the range of ordinary expectations, and should encompass the variety of products or applications for which the model is intended.',
 'source_documents': [Document(page_content='Version 1.0 \nTesting \nSupervisory Guidance on Model Risk Management \nAn integral part of model development is testing, in which the various components of a model and \nits overall functioning are evaluated to determine whether the model is performing as intended. \nModel testing includes checking the model’s accuracy, demonstrating that the model is robust and \nstable, assessing potential limitations, and evaluating the model’s behavior over a range of input \nvalues. It should also assess the impact of a

In [ ]:
### the correct answers can also be provided via a csv file
correct_answers_csv = pd.read_csv('correct-answers.csv')
correct_answers_csv

#### Handwritten Test

In [112]:
for prompt in number_prompts.data:
  print(prompt)

The number of testing for marketing model is  
The number of testing for CCAR model is  


In [113]:
def generate_response(prompts: List[str]) -> List[str]:
    sentences = []
    for prompt in prompts:
      ans = qa_chain(prompt)
      # print(ans)
      sentences.append(ans['result'])
    return sentences


# generate_response(number_prompts.data)

In [114]:
prompts = pd.DataFrame({"id": [], "prompt": []})
responses = pd.DataFrame({"id": [], "response": []})
test_results = pd.DataFrame({"id": [], "p/f": []})

model_responses = generate_response(number_prompts.data)

for (i, response) in enumerate(model_responses):
    pf = 'fail'
    model_type = number_prompts.meta[i]["model_type"]
    
    # Check if the correct language is in the response
    number = correct_responses[model_type]
    if number in response:
        pf = 'pass'

    prompts = prompts.append({"id": i, "prompt": number_prompts.data[i]}, ignore_index=True)
    responses = responses.append({"id": i, "response": response}, ignore_index=True)
    test_results = test_results.append({"id": i, "p/f": pf}, ignore_index=True)

#### Show test results
Let's look at our test results. The first dataframe contains the prompts given to the model.

In [115]:
prompts

,id,prompt
0,0.0,The number of testing for marketing model is
1,1.0,The number of testing for CCAR model is


In [116]:
responses

,id,response
0,0.0,The text does not provide a specific number fo...
1,1.0,The given context does not provide information...


In [117]:
test_results

,id,p/f
0,0.0,fail
1,1.0,fail


#### Testing with Checklist


Next, let's try running the MFT with Checklist. We will no longer need to keep track of results in Pandas dataframes, since Checklist will track the results for us.

#### Create the expectation function
In order to determine if an example passes or fails the test, Checklist uses an expectation function. An expectation function is a function that receives the example, then returns true if the example passes the test, or false if the example fails.

In [118]:
# TODO: Need to test if it will work with differnt formats, i.e., 3 numeric, 3 character, 3 string, three.
def response_contains_correct_number(x, pred, conf, label=None, meta=None):
    model_type = meta['model_type']
    number = correct_responses[model_type]
    return number in pred

We will wrap this function with `Expect.single`, which causes the expectation function to be called for each example. In other cases, you might want to have an expectation function that checks multiple examples simulatneously. See the tutorial notebook "3. Test types, expectation functions, running tests" for detailed information about expectation functions.

In [119]:
correct_number_expect_fn = Expect.single(response_contains_correct_number)

Now we can feed our prompts and expectation function into the MFT constructor.

In [120]:
test = MFT(**number_prompts, name='Correct number in response', description='The response contains the correct number of testings in the prompt.', expect=correct_number_expect_fn)

In order to run the test, Checklist also needs a function that generates the model's predictions for the inputs. The function receives all inputs (prompts) as a list, and must return the results in a tuple `(model_predictions, confidences)`, where `model_predictions` is a list of all the predictions, and `confidences` is a list of the model's scores for those predictions.

We will not be using confidences in this test. Checklist provides a wrapper function `PredictorWrapper.wrap_predict()` that outputs a tuple with a confidence score of 1 for any prediction. We can use it to wrap `generate_sentences` so the predictions will have a confidence score as needed.

In [121]:
wrapped_generator = PredictorWrapper.wrap_predict(generate_response)
# wrapped_generator(["The number of testing for marketing model is"])

In [122]:
test.run(wrapped_generator, overwrite=True)

Predicting 2 examples


In [123]:
def format_example(x, pred, conf, label=None, meta=None): 
    return 'Prompt:      %s\nCompletion:      %s' % (x, pred) 

In [124]:
test.summary(format_example_fn = format_example)

Test cases:      2
Fails (rate):    2 (100.0%)

Example fails:
Prompt:      The number of testing for marketing model is  
Completion:      The text does not provide a specific number for the amount of testing required for a marketing model. It states that testing should be applied to actual circumstances under a variety of market conditions, including scenarios that are outside the range of ordinary expectations, and should encompass the variety of products or applications for which the model is intended.
----
Prompt:      The number of testing for CCAR model is  
Completion:      The given context does not provide information about the specific number of testing for CCAR model.
----


In [160]:
from google.colab import output
output.enable_custom_widget_manager()

In [161]:
test.visual_summary()

TestSummarizer(stats={'npassed': 0, 'nfailed': 2, 'nfiltered': 0}, summarizer={'name': None, 'description': No…

### INVariance: adding irrelevant stuff before and after
You can either use existing perturbation (11) comes with the perturb class, i.e., strip_punctuation, add_typos, remove_negation, etc. 
Or write you own perturb function. Examples can be found at `2. Perturbing data.ipynb`. 

I am doing one as an example for our chatbot, i.e, add irrelevant stuff. 
One can also add functions to the perturb class and update the checklist repo through a PR if the owner approves.

In [148]:
import string
import numpy as np
def random_string(n):
    return ''.join(np.random.choice([x for x in string.ascii_letters + string.digits], n))
# def random_url(n=6):
    # return 'https://t.co/%s' % random_string(n)
# def random_handle(n=6):
    # return '@%s' % random_string(n)

# data['sentence']

def add_irrelevant(sentence):
    # urls_and_handles = [random_url(n=6) for _ in range(5)] + [random_handle() for _ in range(5)]
    # irrelevant_before = ['@airline '] + urls_and_handles
    irrelevant_before = [random_string(n=6) for _ in range(1)]
    # irrelevant_after = urls_and_handles 
    irrelevant_after = [random_string(n=10) for _ in range(1)]
    rets = ['%s %s' % (x, sentence) for x in irrelevant_before ]
    rets += ['%s %s' % (sentence, x) for x in irrelevant_after]
    return rets

In [149]:
suite = TestSuite()
sentences=number_prompts.data

In [150]:
t = Perturb.perturb(sentences, add_irrelevant, nsamples=2)
test = INV(t.data)
suite.add(test, 'add random chars', 'Robustness', 'add randomly generated chars to the start or end of sentence')

In [151]:
t.data

[['The number of testing for marketing model is  ',
  'GOChwB The number of testing for marketing model is  ',
  'The number of testing for marketing model is   FLHy5LJTSG'],
 ['The number of testing for CCAR model is  ',
  'UoPpzA The number of testing for CCAR model is  ',
  'The number of testing for CCAR model is   uhGXeHgPSQ']]

In [152]:
suite.run(wrapped_generator, overwrite=True)

Running add random chars
Predicting 6 examples


In [153]:
suite.summary(format_example_fn = format_example)

Robustness

add random chars
Test cases:      2
Fails (rate):    2 (100.0%)

Example fails:
Prompt:      The number of testing for marketing model is  
Completion:      The text does not provide a specific number for the testing of marketing models. It states that testing should be applied to actual circumstances under a variety of market conditions, including scenarios that are outside the range of ordinary expectations, and should encompass the variety of products or applications for which the model is intended.
Prompt:      GOChwB The number of testing for marketing model is  
Completion:      I'm sorry, but the given context does not provide information about the specific number of testing for marketing models. It only explains the importance of testing in model development and the various components that should be evaluated during testing.
Prompt:      The number of testing for marketing model is   FLHy5LJTSG
Completion:      I'm sorry, but I cannot find any information related to

AttributeError: ignored

### By the way, test suit can be outputed and loaded later. Similary, prediction can also be saved and used later.
### Using files with test suites

Some models cannot be run directly on the same machine that is running the Checklist test suite. For instance, a model might need to run in a specially configured lab environment. In this case, Checklist does not have to receive the predictions from the model directly. The predictions can be saved to a file, then the test suite can check the predictions from the file.



#### Exporting a test suite to a file
First, let's create a file that contains all the prompts that we will send to the model. TestSuite's `to_raw_file()` function exports a test suite to a file.

In [154]:
suite.to_raw_file("suite.txt")

In [156]:
cat "suite.txt"

The number of testing for marketing model is  
GOChwB The number of testing for marketing model is  
The number of testing for marketing model is   FLHy5LJTSG
The number of testing for CCAR model is  
UoPpzA The number of testing for CCAR model is  
The number of testing for CCAR model is   uhGXeHgPSQ

#### Generating the predictions
Next, we need to generate predictions and save them to another file named `suite_predictions.txt`. By default, Checklist expects to receive 1 prediction per line, so we will replace any newline characters with spaces in each prediction.

In [162]:
with open('suite.txt', 'r') as input_file:
    with open('suite_predictions.txt', 'w') as output_file:
        for line in input_file:
            prediction = generate_response(line)
            prediction = prediction.replace('\n', ' ')
            print(prediction, file=output_file)

  "error": {
    "message": "The server had an error while processing your request. Sorry about that! You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 2440937a19b47de057cf5521ce424594 in your message.)",
    "type": "server_error",
    "param": null,
    "code": null
  }
}
 500 {'error': {'message': 'The server had an error while processing your request. Sorry about that! You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 2440937a19b47de057cf5521ce424594 in your message.)', 'type': 'server_error', 'param': None, 'code': None}} {'Date': 'Tue, 23 May 2023 00:02:41 GMT', 'Content-Type': 'application/json', 'Content-Length': '366', 'Connection': 'keep-alive', 'access-control-allow-origin': '*', 'openai-organization': 'user-b4fjhuccihix3ujnlvek2e0q', 'openai-processing-ms': '30012', 'openai-version': '2020-10-01', '

AttributeError: ignored

In [ ]:
cat "suite_predictions.txt"

#### Run from file

In [ ]:
suite.run_from_file("suite_predictions.txt", file_format="pred_only", overwrite=True)

In [ ]:
suite.summary(format_example_fn = format_example)

### DIV Directional Expectation tests

## To Delete Make a chain

In [ ]:
# create the chain to answer questions 
qa_chain = RetrievalQA.from_chain_type(llm=OpenAI(), 
                                  chain_type="stuff", 
                                  retriever=retriever, 
                                  return_source_documents=True)

In [ ]:
## Cite sources
def process_llm_response(llm_response):
    print(llm_response['result'])
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

In [ ]:
# full example
query = "What is a model?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 A model is a simplified representation of real-world relationships among observed characteristics, values, and events.


Sources:
/content/drive/MyDrive/LLM/data/pub-ch-model-risk.pdf
/content/drive/MyDrive/LLM/data/fil17022a.pdf


In [ ]:
# break it down
query = "How to evalue model performance?"
llm_response = qa_chain(query)
# process_llm_response(llm_response)
llm_response

{'query': 'How to evalue model performance?',
 'result': ' Model performance can be evaluated by comparing model estimates and outputs to actual outcomes, assessing the reasons for observed variation between the two, using appropriate tests such as assessment of the accuracy of estimates or forecasts or an evaluation of rank- order ability, using multiple tests since one individual test will have weaknesses, evaluating the quality and extent of developmental evidence, providing appropriate critical review of model selection and development, determining if the model reflects sound theory and business practice, comparing alternative theories and approaches, verifying model stability and accuracy, assessing key assumptions and variables, evaluating the relevance and sufficiency of the data used to build the model, verifying the effectiveness of model performance with respect to the range of model inputs and assumptions, and assessing whether biases are present in data and model outcomes.'

In [ ]:
# query = "Who led the round in Pando?"
# llm_response = qa_chain(query)
# process_llm_response(llm_response)

 Iron Pillar and Uncorrelated Ventures.


Sources:
new_articles/05-03-ai-powered-supply-chain-startup-pando-lands-30m-investment.txt
new_articles/05-03-ai-powered-supply-chain-startup-pando-lands-30m-investment.txt


In [ ]:
# query = "What did databricks acquire?"
# llm_response = qa_chain(query)
# process_llm_response(llm_response)

 Databricks acquired Okera, a data governance platform with a focus on AI.


Sources:
new_articles/05-03-databricks-acquires-ai-centric-data-governance-platform-okera.txt
new_articles/05-03-databricks-acquires-ai-centric-data-governance-platform-okera.txt


In [ ]:
# query = "What is generative ai?"
# llm_response = qa_chain(query)
# process_llm_response(llm_response)

 Generative AI is a type of artificial intelligence that is used to create new content associated with a company, such as content for a website or ads. It can also be used to automate processes and workflows.


Sources:
new_articles/05-04-slack-updates-aim-to-put-ai-at-the-center-of-the-user-experience.txt
new_articles/05-03-nova-is-building-guardrails-for-generative-ai-content-to-protect-brand-integrity.txt


In [ ]:
# query = "Who is CMA?"
# llm_response = qa_chain(query)
# process_llm_response(llm_response)

 The CMA stands for the Competition and Markets Authority.


Sources:
new_articles/05-04-cma-generative-ai-review.txt
new_articles/05-04-cma-generative-ai-review.txt


In [ ]:
qa_chain.retriever.search_type , qa_chain.retriever.vectorstore

('similarity', <langchain.vectorstores.chroma.Chroma at 0x7f8ea02d21d0>)

In [ ]:
print(qa_chain.combine_documents_chain.llm_chain.prompt.template)

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Helpful Answer:


## To Delete ConversationalRetrievalChain

## To Delete Deleteing the DB

In [ ]:
!zip -r db.zip ./db

  adding: db/ (stored 0%)
  adding: db/chroma-collections.parquet (deflated 50%)
  adding: db/index/ (stored 0%)
  adding: db/index/index_metadata_59c51927-205d-4fd7-88d8-c7ba851bd2a5.pkl (deflated 5%)
  adding: db/index/uuid_to_id_59c51927-205d-4fd7-88d8-c7ba851bd2a5.pkl (deflated 39%)
  adding: db/index/index_59c51927-205d-4fd7-88d8-c7ba851bd2a5.bin (deflated 17%)
  adding: db/index/id_to_uuid_59c51927-205d-4fd7-88d8-c7ba851bd2a5.pkl (deflated 35%)
  adding: db/chroma-embeddings.parquet (deflated 29%)


In [ ]:
# To cleanup, you can delete the collection
vectordb.delete_collection()
vectordb.persist()

# delete the directory
!rm -rf db/

## To Delete Starting again loading the db

restart the runtime

In [ ]:
!unzip db.zip

Archive:  db.zip
   creating: db/
  inflating: db/chroma-collections.parquet  
   creating: db/index/
  inflating: db/index/index_metadata_59c51927-205d-4fd7-88d8-c7ba851bd2a5.pkl  
  inflating: db/index/uuid_to_id_59c51927-205d-4fd7-88d8-c7ba851bd2a5.pkl  
  inflating: db/index/index_59c51927-205d-4fd7-88d8-c7ba851bd2a5.bin  
  inflating: db/index/id_to_uuid_59c51927-205d-4fd7-88d8-c7ba851bd2a5.pkl  
  inflating: db/chroma-embeddings.parquet  


In [ ]:
import os

os.environ["OPENAI_API_KEY"] = ""

In [ ]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings

from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

In [ ]:
persist_directory = 'db'
embedding = OpenAIEmbeddings()

vectordb2 = Chroma(persist_directory=persist_directory, 
                  embedding_function=embedding,
                   )

retriever = vectordb2.as_retriever(search_kwargs={"k": 2})

In [ ]:
# Set up the turbo LLM
turbo_llm = ChatOpenAI(
    temperature=0,
    model_name='gpt-3.5-turbo'
)

In [ ]:
# create the chain to answer questions 
qa_chain = RetrievalQA.from_chain_type(llm=turbo_llm, 
                                  chain_type="stuff", 
                                  retriever=retriever, 
                                  return_source_documents=True)

In [ ]:
## Cite sources
def process_llm_response(llm_response):
    print(llm_response['result'])
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

In [ ]:
# full example
query = "How much money did Pando raise?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

Pando raised $30 million in a Series B round, bringing its total raised to $45 million.


Sources:
new_articles/05-03-ai-powered-supply-chain-startup-pando-lands-30m-investment.txt
new_articles/05-03-ai-powered-supply-chain-startup-pando-lands-30m-investment.txt


### Chat prompts

In [ ]:
print(qa_chain.combine_documents_chain.llm_chain.prompt.messages[0].prompt.template)

Use the following pieces of context to answer the users question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
{context}


In [ ]:
print(qa_chain.combine_documents_chain.llm_chain.prompt.messages[1].prompt.template)

{question}
